In [1]:
def unique(list1):
 
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    # for x in unique_list:
    #     print x,
    return(unique_list)

In [2]:
from datasets import load_dataset
dataset = load_dataset("gretelai/symptom_to_diagnosis")

train = dataset["train"]
train_label = train[:]["output_text"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
labels=(unique(train_label))
labels_dict = {labels[i]: i for i in range(len(labels))}
print(labels_dict)

{'typhoid': 0, 'fungal infection': 1, 'jaundice': 2, 'allergy': 3, 'varicose veins': 4, 'diabetes': 5, 'malaria': 6, 'hypertension': 7, 'migraine': 8, 'urinary tract infection': 9, 'arthritis': 10, 'gastroesophageal reflux disease': 11, 'impetigo': 12, 'chicken pox': 13, 'peptic ulcer disease': 14, 'pneumonia': 15, 'psoriasis': 16, 'cervical spondylosis': 17, 'dengue': 18, 'bronchial asthma': 19, 'drug reaction': 20, 'common cold': 21}


In [5]:
batch_size = 16 # can experiment with different sizes
dataset = dataset.rename_column("output_text", "labels")
dataset = dataset.class_encode_column("labels")
dataset = dataset.rename_column("input_text", "text")
# dataset = dataset.map(input_columns="output_text", remove_columns="output_text", batched=True, batch_size=batch_size)
print(dataset)

Casting to class labels:   0%|          | 0/853 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/212 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 853
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 212
    })
})


In [6]:
print(dataset["train"].features)

{'labels': ClassLabel(names=['allergy', 'arthritis', 'bronchial asthma', 'cervical spondylosis', 'chicken pox', 'common cold', 'dengue', 'diabetes', 'drug reaction', 'fungal infection', 'gastroesophageal reflux disease', 'hypertension', 'impetigo', 'jaundice', 'malaria', 'migraine', 'peptic ulcer disease', 'pneumonia', 'psoriasis', 'typhoid', 'urinary tract infection', 'varicose veins'], id=None), 'text': Value(dtype='string', id=None)}


In [7]:
train = dataset["train"]
# train_label = train[:]["label"]
# print(train_label)

# Split the 10% test + valid in half test, half valid
# test_valid = dataset["test"].train_test_split(0.5)

# train_test_valid_dataset = DatasetDict({
#     'train': train,
#     'test': test_valid['test'],
#     'validation': test_valid['train']})

train_test_valid_dataset = dataset

print(train_test_valid_dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 853
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 212
    })
})


In [8]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)

def preprocess(sample):
    return tokenizer(sample["text"], truncation=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
tokenized_dataset = train_test_valid_dataset.map(preprocess, batched=True, batch_size=batch_size)
print(tokenized_dataset)


Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 853
    })
    test: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 212
    })
})


In [10]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# DataCollatorForTokenClassification

2024-05-02 23:27:21.316629: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 23:27:25.846486: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
from evaluate import load
metric = load('accuracy')

In [12]:
import numpy as np

def compute_metrics(preds):
    logits, labels = preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# batch_size = 110 # can experiment with different sizes

args = TrainingArguments(
    "test", # directory to save the model
    evaluation_strategy = "epoch", # evaluate after each epoch
    save_strategy = "epoch", # save after each epoch
    learning_rate=2e-5, # the learning rate to use
    per_device_train_batch_size=batch_size, # the batch size
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20, # number of epochs; 5 took about 30 minutes
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy" # metric associated with COLA GLUE
)

In [28]:
# autoload a model from the base for sequence classification,
# we pass 22 labels
label2id = labels_dict
id2label = {v: k for k, v in labels_dict.items()}
print(id2label)
print(label2id)

{0: 'psoriasis', 1: 'urinary tract infection', 2: 'fungal infection', 3: 'allergy', 4: 'peptic ulcer disease', 5: 'gastroesophageal reflux disease', 6: 'varicose veins', 7: 'hypertension', 8: 'impetigo', 9: 'chicken pox', 10: 'common cold', 11: 'cervical spondylosis', 12: 'bronchial asthma', 13: 'typhoid', 14: 'drug reaction', 15: 'jaundice', 16: 'dengue', 17: 'diabetes', 18: 'pneumonia', 19: 'arthritis', 20: 'malaria', 21: 'migraine'}
{'psoriasis': 0, 'urinary tract infection': 1, 'fungal infection': 2, 'allergy': 3, 'peptic ulcer disease': 4, 'gastroesophageal reflux disease': 5, 'varicose veins': 6, 'hypertension': 7, 'impetigo': 8, 'chicken pox': 9, 'common cold': 10, 'cervical spondylosis': 11, 'bronchial asthma': 12, 'typhoid': 13, 'drug reaction': 14, 'jaundice': 15, 'dengue': 16, 'diabetes': 17, 'pneumonia': 18, 'arthritis': 19, 'malaria': 20, 'migraine': 21}


In [14]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels_dict))

trainer = Trainer(
    model,  # the pre-trained model
    args,  # the TrainingAgruments, defined above
    train_dataset=tokenized_dataset["train"], # the training dataset
    eval_dataset=tokenized_dataset["test"], # the validation dataset
    tokenizer=tokenizer, # our tokenizer
    data_collator=data_collator, # the collator we defined above
    compute_metrics=compute_metrics # our function for computing the metrics
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.763383,0.405660
2,No log,2.124238,0.754717
3,No log,1.542863,0.797170
4,No log,1.133653,0.853774
5,No log,0.818858,0.896226
6,No log,0.580683,0.929245
7,No log,0.428355,0.943396
8,No log,0.320332,0.952830
9,No log,0.249188,0.933962
10,1.210000,0.192927,0.943396


TrainOutput(global_step=1080, training_loss=0.5842200645694027, metrics={'train_runtime': 835.3072, 'train_samples_per_second': 20.424, 'train_steps_per_second': 1.293, 'total_flos': 260102697721620.0, 'train_loss': 0.5842200645694027, 'epoch': 20.0})

In [18]:
# Split the 10% test + valid in half test, half valid
dataset2_train = tokenized_dataset["train"].train_test_split(0.5)
dataset2_test = tokenized_dataset["test"]

from datasets import DatasetDict

dataset2 = DatasetDict({
    'train': dataset2_train["train"],
    'test': dataset2_test})

In [21]:
args2 = TrainingArguments(
    "model_2", # directory to save the model
    evaluation_strategy = "epoch", # evaluate after each epoch
    save_strategy = "no", # save after each epoch
    learning_rate=2e-5, # the learning rate to use
    per_device_train_batch_size=batch_size, # the batch size
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20, # number of epochs; 5 took about 30 minutes
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model="accuracy" # metric associated with COLA GLUE
)

model2 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels_dict))

trainer2 = Trainer(
    model2,  # the pre-trained model
    args2,  # the TrainingAgruments, defined above
    train_dataset=dataset2["train"], # the training dataset
    eval_dataset=dataset2["test"], # the validation dataset
    tokenizer=tokenizer, # our tokenizer
    data_collator=data_collator, # the collator we defined above
    compute_metrics=compute_metrics # our function for computing the metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
trainer2.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.027924,0.165094
2,No log,2.781996,0.382075
3,No log,2.453523,0.514151
4,No log,2.158650,0.603774
5,No log,1.883384,0.698113
6,No log,1.640185,0.754717
7,No log,1.445317,0.806604
8,No log,1.267467,0.830189
9,No log,1.125404,0.839623
10,No log,1.002831,0.867925


Checkpoint destination directory model_2/checkpoint-27 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory model_2/checkpoint-54 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory model_2/checkpoint-81 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory model_2/checkpoint-108 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory model_2/checkpoint-135 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory model_2/checkpoint-162 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory model_2/checkpoint-189 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Ch

TrainOutput(global_step=540, training_loss=1.1813213630958839, metrics={'train_runtime': 597.8993, 'train_samples_per_second': 14.25, 'train_steps_per_second': 0.903, 'total_flos': 129199191688920.0, 'train_loss': 1.1813213630958839, 'epoch': 20.0})

In [22]:
# Split the 10% test + valid in half test, half valid
dataset3_train = dataset2_train["train"].train_test_split(0.5)
dataset3_test = tokenized_dataset["test"]

from datasets import DatasetDict

dataset3 = DatasetDict({
    'train': dataset3_train["train"],
    'test': dataset3_test})

args3 = TrainingArguments(
    "model_3", # directory to save the model
    evaluation_strategy = "epoch", # evaluate after each epoch
    save_strategy = "no", # save after each epoch
    learning_rate=2e-5, # the learning rate to use
    per_device_train_batch_size=batch_size, # the batch size
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20, # number of epochs; 5 took about 30 minutes
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model="accuracy" # metric associated with COLA GLUE
)

model3 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels_dict))

trainer3 = Trainer(
    model3,  # the pre-trained model
    args3,  # the TrainingAgruments, defined above
    train_dataset=dataset3["train"], # the training dataset
    eval_dataset=dataset3["test"], # the validation dataset
    tokenizer=tokenizer, # our tokenizer
    data_collator=data_collator, # the collator we defined above
    compute_metrics=compute_metrics # our function for computing the metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
trainer3.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.063155,0.108491
2,No log,2.989156,0.183962
3,No log,2.854903,0.301887
4,No log,2.706358,0.377358
5,No log,2.559025,0.481132
6,No log,2.424546,0.523585
7,No log,2.289904,0.580189
8,No log,2.167504,0.589623
9,No log,2.065449,0.608491
10,No log,1.968374,0.627358


TrainOutput(global_step=280, training_loss=1.8242047991071428, metrics={'train_runtime': 240.9919, 'train_samples_per_second': 17.677, 'train_steps_per_second': 1.162, 'total_flos': 62451150382392.0, 'train_loss': 1.8242047991071428, 'epoch': 20.0})

In [23]:
# Split the 10% test + valid in half test, half valid
dataset4_train = dataset3_train["train"].train_test_split(0.5)
dataset4_test = tokenized_dataset["test"]

from datasets import DatasetDict

dataset4 = DatasetDict({
    'train': dataset4_train["train"],
    'test': dataset4_test})

args4 = TrainingArguments(
    f"model_4", # directory to save the model
    evaluation_strategy = "epoch", # evaluate after each epoch
    save_strategy = "no", # save after each epoch
    learning_rate=2e-5, # the learning rate to use
    per_device_train_batch_size=batch_size, # the batch size
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20, # number of epochs; 5 took about 40 minutes
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model="accuracy" # metric associated with COLA GLUE
)

model4 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels_dict))

trainer4 = Trainer(
    model4,  # the pre-trained model
    args4,  # the TrainingAgruments, defined above
    train_dataset=dataset4["train"], # the training dataset
    eval_dataset=dataset4["test"], # the validation dataset
    tokenizer=tokenizer, # our tokenizer
    data_collator=data_collator, # the collator we defined above
    compute_metrics=compute_metrics # our function for computing the metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
trainer4.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.090332,0.070755
2,No log,3.081134,0.061321
3,No log,3.069970,0.070755
4,No log,3.047992,0.136792
5,No log,3.021141,0.155660
6,No log,2.977119,0.198113
7,No log,2.928387,0.226415
8,No log,2.881377,0.240566
9,No log,2.835170,0.273585
10,No log,2.783495,0.283019


TrainOutput(global_step=140, training_loss=2.482023184640067, metrics={'train_runtime': 148.0159, 'train_samples_per_second': 14.323, 'train_steps_per_second': 0.946, 'total_flos': 31468087346112.0, 'train_loss': 2.482023184640067, 'epoch': 20.0})

In [28]:
trainer.save_model("/root/.cache/huggingface/diagnosis_model")